In [162]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model, metrics
from sklearn.metrics import (mean_squared_error, r2_score, mean_absolute_error, 
mean_squared_log_error, explained_variance_score, max_error)
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.svm import LinearSVR
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Load training and testing data
X_train = np.loadtxt("X_train.csv", delimiter=',', skiprows=1)
X_test = np.loadtxt("X_test.csv", delimiter=',', skiprows=1)
y_train = np.loadtxt("y_train.csv", delimiter=',', skiprows=1)[:,1]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(168, 6)
(41, 6)
(168,)


In [173]:
# Arrange answer in two columns. First column (with header "Id") is an
# enumeration from 0 to n-1, where n is the number of test points. Second
# column (with header "EpiOrStroma" is the predictions.
def saveFile(y_pred,name):
    test_header = "Id,PRP"
    n_points = X_test.shape[0]
    y_pred_pp = np.ones((n_points, 2))
    y_pred_pp[:, 0] = range(n_points)
    y_pred_pp[:, 1] = y_pred
    np.savetxt(name, y_pred_pp, fmt='%d,%f', delimiter=",",
               header=test_header, comments="")


In [164]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
scaler = StandardScaler()

#scaling 
X_train1_scaled = scaler.fit_transform(X_train1)
X_test1_scaled = scaler.fit_transform(X_test1)

print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

print(X_train1_scaled.shape)
print(X_test1_scaled.shape)

(117, 6)
(51, 6)
(117,)
(51,)
(117, 6)
(51, 6)


In [190]:
# Defining basic untuned regression models

def basicLinearSVR(X_train1, X_test1, y_train1, y_test1):
    svrReg = LinearSVR(max_iter = 9999)
    svrReg.fit(X_train1, y_train1)
    y_pred = svrReg.predict(X_test1)
    print("R2 score: " + str(svrReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [191]:
basicLinearSVR(X_train1_scaled, X_test1_scaled, y_train1, y_test1)

R2 score: 0.673379439257437
Explained variance: 0.7024012709861599
Max error: 238.45169780453432
Mean absolute error: 32.49406435203634
Mean squared error: 4057.944646115132


array([177.84773546,  25.7940557 ,  97.75169166,  36.29708147,
        14.30892991,  11.39655658,  12.57735637,  14.51458585,
        67.36775809,  22.74726369,  50.07263377,  67.42602536,
        55.18276806,  20.80022228,  30.77345661,  39.13318712,
       111.77767581,  44.80792699, 250.75868996,  41.78162852,
         7.788854  ,  66.12466794,  66.01452546,  19.48883275,
        23.95000413,  95.57676713,   0.28963482,  57.80624973,
       226.5483022 ,  63.38306595,  20.80022228,  19.54727247,
        52.97857273,  86.31840841,  29.78950765, 188.97164606,
       177.75674457,  36.09560457,  67.01519158,  85.87628825,
        23.57272307,  47.02638722,  96.76513231,  22.68506095,
       123.16331633,  47.8597801 ,   7.9110046 ,  26.30132087,
       281.85581982,  17.42700323,  -2.22544198])

In [178]:
# Linear Support Vector Regression - Further Tuning with Grid Search
lsvr = LinearSVR()
# Look at parameters used by our regression
print('Parameters currently in use:\n')
print(lsvr.get_params())

#Creating the Random Grid
#pipe = Pipeline([('regression model' , LinearSVR())])

param_grid = [
    {#'regression' : [LinearSVR()],
    'epsilon' : [0,75, 0.5, 0.25, 0],
    'tol' : [1e-4, 2e-4],
     'C' : [0.5, 1.0, 1.5, 2.0],
    'fit_intercept' : [True, False],
    #'intercept_scaling' : [0, 0.5, 1, 1.5, 2],
    #'dual' : [True, False],
    'verbose' : [0, 1],
    'max_iter' : [500, 1000, 1500]}
]


Parameters currently in use:

{'C': 1.0, 'dual': True, 'epsilon': 0.0, 'fit_intercept': True, 'intercept_scaling': 1.0, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'random_state': None, 'tol': 0.0001, 'verbose': 0}


In [168]:
#fit the model with the parameter
reg = GridSearchCV(LinearSVR(), param_grid = param_grid, cv = 10, verbose=True, n_jobs=-1)
# Fit the random search model
best_reg= reg.fit(X_train1_scaled, y_train1)


best_reg.best_params_

Fitting 10 folds for each of 480 candidates, totalling 4800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 1564 tasks      | elapsed:    0.7s


[LibLinear]

[Parallel(n_jobs=-1)]: Done 4800 out of 4800 | elapsed:    1.2s finished
/home/nicholas/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'C': 2.0,
 'epsilon': 0.5,
 'fit_intercept': True,
 'max_iter': 500,
 'tol': 0.0001,
 'verbose': 1}

In [169]:
# tuned regression models

def tunedLinearSVR(X_train1, X_test1, y_train1, y_test1):
    svrReg = LinearSVR(C=2.0, epsilon= 0.5, fit_intercept = True, max_iter = 500, tol = 0.0001, verbose = 1 )
    svrReg.fit(X_train1, y_train1)
    y_pred = svrReg.predict(X_test1)
    print("R2 score: " + str(svrReg.score(X_test1, y_test1)))
    print("Explained variance: " + str(explained_variance_score(y_test1, y_pred)))
    print("Max error: " + str(max_error(y_test1, y_pred)))
    print("Mean absolute error: " + str(mean_absolute_error(y_test1, y_pred)))
    print("Mean squared error: " + str(mean_squared_error(y_test1, y_pred)))
    return y_pred

In [171]:
y_pred = tunedLinearSVR(X_train1_scaled, X_test1_scaled, y_train1, y_test1)


[LibLinear]R2 score: 0.8152736106552033
Explained variance: 0.8152967088025329
Max error: 139.16187753786505
Mean absolute error: 32.55002427152246
Mean squared error: 2295.046769051159


In [194]:
svrReg = LinearSVR(C=2.0, epsilon= 0.5, fit_intercept = True, max_iter = 5000, tol = 0.0001, verbose = 1 )
svrReg.fit(X_train, y_train)
y_pred = svrReg.predict(X_test)
saveFile(y_pred,'svr_tuned_scaled.csv')

[LibLinear]

/home/nicholas/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
